In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from collections import Counter
import os

import torch
from torchvision import datasets,transforms,models
from torch.utils.data import Dataset,DataLoader
from PIL import Image

import sys
import torch.optim as optim
import time
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as td
import random
import torchvision

In [3]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import xmltodict

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
img_names=[] 
xml_names=[] 
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Colab Notebooks/archive/'):
    for filename in filenames:
        if os.path.join(dirname, filename)[-3:]!="xml":
            img_names.append(filename)
        else:
            xml_names.append(filename)

In [7]:
path_annotations="/content/drive/MyDrive/Colab Notebooks/archive/annotations/" 
listing=[]
for i in img_names[:]:
    with open(path_annotations+i[:-4]+".xml") as fd:
        doc=xmltodict.parse(fd.read())
    temp=doc["annotation"]["object"]
    if type(temp)==list:
        for i in range(len(temp)):
            listing.append(temp[i]["name"])
    else:
        listing.append(temp["name"])
        

Items = Counter(listing).keys()
values = Counter(listing).values()
print(Items,'\n',values)

dict_keys(['without_mask', 'with_mask', 'mask_weared_incorrect']) 
 dict_values([717, 3232, 123])


In [ ]:
path_image="/content/drive/MyDrive/Colab Notebooks/archive/images/"  
def face_cas(img): 
    add = True
    with open(path_annotations+img[:-4]+".xml") as fd:
        doc=xmltodict.parse(fd.read())
    image=plt.imread(os.path.join(path_image+img))
    fig,ax=plt.subplots(1)
    ax.axis("off")
    fig.set_size_inches(10,5)
    temp=doc["annotation"]["object"]
    mpatch = None;
    if type(temp)==list:
        for i in range(len(temp)):
            ###with_mask
            if temp[i]["name"]=="with_mask":
                x,y,w,h=list(map(int,temp[i]["bndbox"].values()))
                mpatch=mpatches.Rectangle((x,y),w-x,h-y,linewidth=1, edgecolor='g',facecolor="none",lw=2,)
                ax.add_patch(mpatch)
                rx, ry = mpatch.get_xy()
                ax.annotate("with_mask", (rx, ry), color='green', weight='bold', fontsize=10, ha='left', va='baseline')
            ###without_mask
            if temp[i]["name"]=="without_mask":
                x,y,w,h=list(map(int,temp[i]["bndbox"].values()))     
                mpatch=mpatches.Rectangle((x,y),w-x,h-y,linewidth=1, edgecolor='r',facecolor="none",lw=2,)
                ax.add_patch(mpatch)
                rx, ry = mpatch.get_xy()
                ax.annotate("without_mask", (rx, ry), color='red', weight='bold', fontsize=10, ha='left', va='baseline')
            ###mask_weared_incorrect
            if temp[i]["name"]=="mask_weared_incorrect":
               pass
    else:
        x,y,w,h=list(map(int,temp["bndbox"].values()))
        edgecolor={"with_mask":"g","without_mask":"r"}
      
        if temp["name"] != "mask_weared_incorrect":
          add = False
          mpatch=mpatches.Rectangle((x,y),w-x,h-y,linewidth=1, edgecolor=edgecolor[temp["name"]],facecolor="none",)
    ax.imshow(image)
    if add and mpatch:
      ax.add_patch(mpatch)

fun_images = img_names.copy()
for i in range(1,8):
    face_cas(fun_images[i])

In [ ]:
options={"with_mask":0,"without_mask":1,"mask_weared_incorrect":2} 
def dataset_creation(image_list): 
    image_tensor=[]
    label_tensor=[]
    for i,j in enumerate(image_list):
        with open(path_annotations+j[:-4]+".xml") as fd:
            doc=xmltodict.parse(fd.read())
        if type(doc["annotation"]["object"])!=list:
            temp=doc["annotation"]["object"]
            x,y,w,h=list(map(int,temp["bndbox"].values()))
            if temp["name"] != "mask_weared_incorrect":
              label=options[temp["name"]]
              image=transforms.functional.crop(Image.open(path_image+j).convert("RGB"), y,x,h-y,w-x)
              image_tensor.append(my_transform(image))
              label_tensor.append(torch.tensor(label))
        else:
            temp=doc["annotation"]["object"]
            for k in range(len(temp)):
                x,y,w,h=list(map(int,temp[k]["bndbox"].values()))
                if temp[k]["name"] != "mask_weared_incorrect":
                  label=options[temp[k]["name"]]
                  image=transforms.functional.crop(Image.open(path_image+j).convert("RGB"),y,x,h-y,w-x)
                  image_tensor.append(my_transform(image))
                  label_tensor.append(torch.tensor(label))
                
    final_dataset=[[k,l] for k,l in zip(image_tensor,label_tensor)]
    return tuple(final_dataset)


my_transform=transforms.Compose([transforms.Resize((64, 64)),
                                 transforms.ToTensor()])

mydataset=dataset_creation(img_names)

In [ ]:
train_size=int(len(mydataset)*0.7)
test_size=len(mydataset)-train_size
print('Length of dataset is', len(mydataset), '\nLength of training set is :',train_size,'\nLength of test set is :', test_size)
trainset,testset=torch.utils.data.random_split(mydataset,[train_size,test_size])

In [ ]:
train_dataloader =DataLoader(dataset=trainset,batch_size=32,shuffle=True,num_workers=4)
test_dataloader =DataLoader(dataset=testset,batch_size=32,shuffle=True,num_workers=4)

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available()
                               else "cpu")

In [ ]:
import sys
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:


nc = 3
nz = 100
ngf = 64
ndf = 64
epochs = 10
g_lr = 0.0001
d_lr = 0.0004
beta1 = 0.5
ngpu=1

def initialize_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)



class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
            return output

generator = Generator(ngpu).to(device)
generator.apply(initialize_weights)
print(generator)

discriminator = Discriminator(ngpu).to(device)
discriminator.apply(initialize_weights)
print(discriminator)

In [ ]:
epochs = 10
real_label = 0.9
fake_label = 0.1
optimizerD = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
g_loss = []
d_loss = []
loss_func = nn.BCELoss()
iters = 0
imgs = []
train_accuracy = []
test_loss = []
test_accuracy = []

for epoch in range(epochs):
    test_correct = 0
    test_total = 0
    for i, data in enumerate(train_dataloader, 0):
  
        # train with real
        discriminator.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = discriminator(real_cpu).view(-1)
        errD_real = loss_func(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = generator(noise)
        label.fill_(fake_label)
        output = discriminator(fake.detach()).view(-1)
        errD_fake = loss_func(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        
        generator.zero_grad()
        label.fill_(real_label)  
        output = discriminator(fake).view(-1)
        errG = loss_func(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        #get train accuracy
        image_data, label_data = data[0], data[1]
        image_data, label_data = image_data.to(device), label_data.to(device)
        discriminator.zero_grad()
        real_cpu = images.to(device)
        total += label_data.size(0)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)
        output = discriminator(real_cpu).view(-1)
        output = output.to(device)
        correct += (output.view(-1).long() == label_data).sum()


        if (iters % 500 == 0) or ((epoch == epochs-1) and (i == len(train_dataloader)-1)):
          with torch.no_grad():
            fixed_noise = torch.randn(ngf, nz, 1, 1, device=device)
            fake_display = generator(fixed_noise).detach().cpu()
            imgs.append(torchvision.utils.make_grid(fake_display, padding=2, normalize=True))


        iters += 1

    accuracy = 100 * correct.float() / total
    accuracy = accuracy.min().cpu().detach().numpy().tolist()
    train_accuracy.append(accuracy)

    g_loss.append(errG.item())
    d_loss.append(errD.item())  

    correct = 0
    total = 0
    
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        discriminator.zero_grad()
        real_cpu = images.to(device)
        total += labels.size(0)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)
        output = discriminator(real_cpu).view(-1)
        output = output.to(device)
        test_error = loss_func(output, labels.float())
        correct += (output.view(-1).long() == labels).sum()

      
    accuracy = 100 * correct.float() / total
    accuracy = accuracy.min().cpu().detach().numpy().tolist()
    test_accuracy.append(accuracy)
    test_loss.append(test_error.item())

    print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f' % (epoch+1, epochs, errD.item(), errG.item()))        

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(g_loss,label="Generator Loss")
plt.plot(d_loss,label="Discriminator Loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
print(d_loss)
print(test_loss)
print(train_accuracy)
print(test_accuracy)

In [ ]:
!pip install torchmetrics

In [ ]:
from torchmetrics.functional import precision_recall
correct = 0
total = 0
total_precision = 0
total_recall = 0
loss = []
for epoch in range(epochs):
    for images, labels in test_dataloader:
          images, labels = images.to(device), labels.to(device)
          discriminator.zero_grad()
          real_cpu = images.to(device)
          total += labels.size(0)
          batch_size = real_cpu.size(0)
          label = torch.full((batch_size,), real_label, device=device)
          output = discriminator(real_cpu).view(-1)
          output = output.to(device)
          test_error = loss_func(output, labels)
          correct += (output.view(-1).long() == labels).sum()
          total_precision += precision_recall(output, labels, average='macro', num_classes=1)[0]
          total_recall += precision_recall(output, labels, average='macro', num_classes=1)[1]
    loss.append(test_error.item())

accuracy = 100 * correct.float() / total
precision = total_precision / total
recall = total_recall / total